In [ ]:
%load_ext autoreload
%autoreload 2

## add to libs

## install requirements

* maybe I should use a fresh venv copied from ai base for each project?

In [ ]:
!pip install -qq -U fastai ipywidgets pillow pillow-avif-plugin inflect
!pip install -qq -U selenium webdriver_manager retry
!pip install -qq -U duckduckgo_search
!pip install -qq -U clip-retrieval

In [ ]:
!jupyter nbextension enable --py --sys-prefix widgetsnbextension

Enabling notebook extension jupyter-js-widgets/extension...
      - Validating: OK


In [ ]:
import pillow_avif
from fastai.vision.all import *
from send2trash import send2trash

In [ ]:
from duckduckgo_search import ddg_images
from clip_retrieval.clip_client import ClipClient

In [ ]:
sys.path.append('.')

In [ ]:
from sam_misc import *
from google_images import install_webdriver, start_chrome, google_image_search

## general setup

In [ ]:
data = Path("bird_cat_dog")

In [ ]:
null_query = "photo of outdoors"
labels = ["bird", "cat", "dog"]
query_prefix = "photo of "

In [ ]:
samples_per_query = 200

## 1. find images using Google search

In [ ]:
searcher = "gimg"

In [ ]:
confirm_delete(data/searcher)

Button(description='Move data to trash: bird_cat_dog/gimg?', layout=Layout(width='20em'), style=ButtonStyle())

In [ ]:
with start_chrome() as wd:
    for comb in powerset(labels):
        others = seq_diff(labels, comb)
        dirname = "_".join(comb) + "_"
        path = data/searcher/dirname
        query = query_prefix + join_a_foo_and_a_bar(comb) if comb else null_query
        query += " " + " ".join("-"+x for x in [*others, "stock"])
        try:
            path.mkdir(parents=True, exist_ok=False)
        except FileExistsError as e:
            print(f"already downloaded: {query}")
            continue
        print(f"downloading: {query}")
        # want any = CC to avoid stock photos, but not many CC images have all three
        opts = "tbs=il:cl" if len(comb) < 2 else ''
        urls = google_image_search(query, safe=True, n=samples_per_query, opts=opts, wd=wd)
        download_images(dest=path, urls=urls)

INFO:WDM:====== WebDriver manager ======
INFO:WDM:Get LATEST chromedriver version for google-chrome 111.0.5563
INFO:WDM:Driver [/home/sam/.wdm/drivers/chromedriver/linux64/111.0.5563/chromedriver] found in cache


downloading: photo of outdoors -bird -cat -dog -stock
downloading: photo of a bird -cat -dog -stock
downloading: photo of a cat -bird -dog -stock
downloading: photo of a dog -bird -cat -stock
downloading: photo of a bird and a cat -dog -stock
downloading: photo of a bird and a dog -cat -stock
downloading: photo of a cat and a dog -bird -stock
downloading: photo of a bird and a cat and a dog -stock


## 2. find images using duckduckgo_search

In [ ]:
searcher = "ddg"

In [ ]:
confirm_delete(data/searcher)

Button(description='Move data to trash: bird_cat_dog/ddg?', layout=Layout(width='20em'), style=ButtonStyle())

In [ ]:
for comb in powerset(labels):
    others = seq_diff(labels, comb)
    dirname = "_".join(comb) + "_"
    path = data/searcher/dirname
    query = query_prefix + join_a_foo_and_a_bar(comb) if comb else null_query
    query += " " + " ".join("-"+x for x in others)
    try:
        path.mkdir(parents=True, exist_ok=False)
    except FileExistsError as e:
        print(f"already downloaded: {query}")
        continue
    print(f"downloading: {query}")
    # want any = CC to avoid stock photos, but not many CC images have all three
    license = "any" if len(comb) < 2 else None
    results = ddg_images(query, max_results=samples_per_query, license_image=license)
    urls = [r["image"] for r in results]
    download_images(dest=path, urls=urls)

already downloaded: photo of outdoors -bird -cat -dog
already downloaded: photo of a bird -cat -dog
already downloaded: photo of a cat -bird -dog
already downloaded: photo of a dog -bird -cat
already downloaded: photo of a bird and a cat -dog
already downloaded: photo of a bird and a dog -cat
already downloaded: photo of a cat and a dog -bird
already downloaded: photo of a bird and a cat and a dog 


## 3. find images using laion

Deduplication means that fewer than `samples_per_query` images will be returned, around 75% or so.

In [ ]:
searcher = "laion"

In [ ]:
confirm_delete(data/searcher)

Button(description='Move data to trash: bird_cat_dog/laion?', layout=Layout(width='20em'), style=ButtonStyle()…

In [ ]:
laion = ClipClient(
    url="https://knn.laion.ai/knn-service",
    indice_name="laion5B-H-14",
    aesthetic_score=0, aesthetic_weight=0,
    num_images=samples_per_query)

In [ ]:
for comb in powerset(labels):
    dirname = "_".join(comb) + "_"
    path = data/searcher/dirname
    query = query_prefix + ", ".join(comb) if comb else null_query
    try:
        path.mkdir(parents=True, exist_ok=False)
    except FileExistsError as e:
        print(f"already downloaded: {query}")
        continue
    print(f"downloading: {query}")
    results = laion.query(text=query)
    urls = [r["url"] for r in results]
    download_images(dest=path, urls=urls)

downloading: photo of outdoors
downloading: photo of bird
downloading: photo of cat
downloading: photo of dog
downloading: photo of bird, cat
downloading: photo of bird, dog
downloading: photo of cat, dog
downloading: photo of bird, cat, dog


### references

- https://replicate.com/blog/grab-hundreds-of-images-with-clip-and-laion
- https://github.com/rom1504/clip-retrieval
- https://rom1504.github.io/clip-retrieval/

## scratch